# 1.0. Imports

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder,StandardScaler, OneHotEncoder, LabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline 

# 2.0. Data Load

In [2]:
data = pd.read_csv('/kaggle/input/inclusao-financeira-na-africa/train.csv')
data.head()
# household / age / gender / cellphone / education_level / job_type = DEIXAR
# marital status / relationship with head? / country? /  location_type? / year?  = TIRAR

,country,year,uniqueid,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type,uid,bank_account
0,Rwanda,2016,uniqueid_4858,Rural,Yes,6,45,Male,Head of Household,Divorced/Seperated,Primary education,Farming and Fishing,Rwanda_uniqueid_4858,No
1,Tanzania,2017,uniqueid_3015,Urban,No,4,33,Female,Head of Household,Single/Never Married,Primary education,Self employed,Tanzania_uniqueid_3015,No
2,Rwanda,2016,uniqueid_103,Rural,Yes,7,43,Male,Head of Household,Married/Living together,Secondary education,Farming and Fishing,Rwanda_uniqueid_103,No
3,Rwanda,2016,uniqueid_4582,Rural,No,6,35,Female,Head of Household,Married/Living together,Primary education,Farming and Fishing,Rwanda_uniqueid_4582,No
4,Tanzania,2017,uniqueid_2854,Urban,Yes,2,30,Male,Head of Household,Single/Never Married,Primary education,Informally employed,Tanzania_uniqueid_2854,No


# 3.0. Initial Data Exploration

In [3]:
data.describe()

,year,household_size,age_of_respondent
count,11762.000000,11762.000000,11762.000000
mean,2016.983336,3.793913,38.602364
std,0.848669,2.225423,16.334624
min,2016.000000,1.000000,16.000000
25%,2016.000000,2.000000,26.000000
50%,2017.000000,3.000000,35.000000
75%,2018.000000,5.000000,48.000000
max,2018.000000,21.000000,100.000000


In [4]:
data.isnull().sum()

country                   0
year                      0
uniqueid                  0
location_type             0
cellphone_access          0
household_size            0
age_of_respondent         0
gender_of_respondent      0
relationship_with_head    0
marital_status            0
education_level           0
job_type                  0
uid                       0
bank_account              0
dtype: int64

In [5]:
data.dtypes

country                   object
year                       int64
uniqueid                  object
location_type             object
cellphone_access          object
household_size             int64
age_of_respondent          int64
gender_of_respondent      object
relationship_with_head    object
marital_status            object
education_level           object
job_type                  object
uid                       object
bank_account              object
dtype: object

In [6]:
# duas categorias - 2 colunas
data.location_type.unique()

array(['Rural', 'Urban'], dtype=object)

In [7]:
# duas categorias
data.cellphone_access.unique()

array(['Yes', 'No'], dtype=object)

In [8]:
# duas categorias
data.gender_of_respondent.unique()

array(['Male', 'Female'], dtype=object)

In [9]:
# 6 categorias - sem ordem
data.relationship_with_head.unique()

array(['Head of Household', 'Spouse', 'Child', 'Parent', 'Other relative',
       'Other non-relatives'], dtype=object)

In [10]:
# 5 categorias - sem ordem
data.marital_status.unique()

array(['Divorced/Seperated', 'Single/Never Married',
       'Married/Living together', 'Widowed', 'Dont know'], dtype=object)

In [11]:
# 6 categorias - com ordem 
data.education_level.unique()

array(['Primary education', 'Secondary education',
       'Vocational/Specialised training', 'Tertiary education',
       'No formal education', 'Other/Dont know/RTA'], dtype=object)

In [12]:
# 10 categorias - sem ordem
data.job_type.unique()

array(['Farming and Fishing', 'Self employed', 'Informally employed',
       'No Income', 'Other Income', 'Remittance Dependent',
       'Formally employed Private', 'Government Dependent',
       'Dont Know/Refuse to answer', 'Formally employed Government'],
      dtype=object)

In [13]:
data.bank_account.value_counts()


No     10077
Yes     1685
Name: bank_account, dtype: int64

In [14]:
print(data['job_type'].value_counts())
print(' ')
print(data['marital_status'].value_counts()) 
print('')
print(data['relationship_with_head'].value_counts())
print('')
print(data['education_level'].value_counts())


Self employed                   3207
Informally employed             2788
Farming and Fishing             2732
Remittance Dependent            1252
Other Income                     569
Formally employed Private        514
No Income                        310
Formally employed Government     206
Government Dependent             125
Dont Know/Refuse to answer        59
Name: job_type, dtype: int64
 
Married/Living together    5433
Single/Never Married       3970
Widowed                    1321
Divorced/Seperated         1034
Dont know                     4
Name: marital_status, dtype: int64

Head of Household      6358
Spouse                 3287
Child                  1099
Parent                  560
Other relative          368
Other non-relatives      90
Name: relationship_with_head, dtype: int64

Primary education                  6408
No formal education                2240
Secondary education                2129
Tertiary education                  566
Vocational/Specialised training

# 4.0. Train Test Split


In [15]:
# Drop id columns - irrelevant for the model
data_dropped = data.drop(['uniqueid', 'uid'], axis=1)

In [16]:
# Separação de x e y 
X = data_dropped.iloc[:,0:11]
y = data_dropped['bank_account']

In [17]:
X.head()

,country,year,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Rwanda,2016,Rural,Yes,6,45,Male,Head of Household,Divorced/Seperated,Primary education,Farming and Fishing
1,Tanzania,2017,Urban,No,4,33,Female,Head of Household,Single/Never Married,Primary education,Self employed
2,Rwanda,2016,Rural,Yes,7,43,Male,Head of Household,Married/Living together,Secondary education,Farming and Fishing
3,Rwanda,2016,Rural,No,6,35,Female,Head of Household,Married/Living together,Primary education,Farming and Fishing
4,Tanzania,2017,Urban,Yes,2,30,Male,Head of Household,Single/Never Married,Primary education,Informally employed


In [18]:
# stratified split - gera indexes
# com o n_splits=1, possivelmente é mesma coisa que o train_test_split 
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=2187)

for train_index, test_index in split.split(X,y):
    X_train, X_val = X.loc[train_index], X.loc[test_index]
    y_train, y_val = y.loc[train_index], y.loc[test_index]

In [19]:
X_train.head()

,country,year,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
9120,Rwanda,2016,Urban,Yes,5,48,Female,Spouse,Married/Living together,Secondary education,Self employed
4584,Kenya,2018,Urban,Yes,1,47,Female,Head of Household,Married/Living together,Secondary education,Formally employed Government
10727,Rwanda,2016,Rural,Yes,2,35,Female,Spouse,Married/Living together,Primary education,Farming and Fishing
7951,Rwanda,2016,Rural,Yes,6,39,Female,Head of Household,Widowed,Primary education,Informally employed
3937,Rwanda,2016,Rural,No,5,44,Female,Spouse,Married/Living together,No formal education,Farming and Fishing


# 5.0. RandomForest - Pipeline

parametros padrão utilizados inicialmente:

Standard:
(n_estimators=100, criterion='gini', max_depth=X, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='sqrt', max_leaf_nodes=X, min_impurity_decrease=0.0, 
bootstrap=True, oob_score=False, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=X)

In [20]:
rf = RandomForestClassifier(random_state=2187)

# parametros random forest (precisa do '__' antes de cada parametro para que o gridsearch entenda que são parametros do modelo
params_rf = {}
params_rf['model__criterion'] = ['entropy', 'gini']
params_rf['model__n_estimators'] = [100, 150]
params_rf['model__max_depth'] = [None,10,15]
params_rf['model__class_weight'] = [None,'balanced', 'balanced_subsample']
# params_rf['model__max_features'] = [None, 'sqrt']
# params_rf['model__min_samples_split'] = [,10,]
# params_rf['model__min_samples_leaf'] = [1,2,] 
params_rf['model'] = [rf]

# StratifiedKFold para que cada fold mantenha a proporção do target
cv_kfold= StratifiedKFold(n_splits=5,random_state=2187,shuffle=True)

# colunas que passarão pelo OneHotEncoding
onehot_columns = ['country', 'relationship_with_head', 'marital_status',  
                  'education_level', 'cellphone_access', 'job_type' , 'location_type', 
                  'gender_of_respondent']

# onehot_columns = ['marital_status','education_level', 'cellphone_access', 'job_type' , 
#                   'gender_of_respondent']

# transformação - passthrough para que as colunas numericas que nao são transformadas pelo one hot passem reto
# dentro de transformers, poderia ter colocado varias transformações diferentes para diferentes colunas
encoders = ColumnTransformer(transformers= [('onehotencod', OneHotEncoder(), onehot_columns)],
                             remainder = 'passthrough', 
                             verbose_feature_names_out = True,
                             verbose=True)

# pipeline - colunas transformadas E colunas numericas entram no pipeline para treinar o modelo
# nos steps, poderia colocar SMOTE() em um dos passos, MinMaxScaler(), StandardScaler(), etc
pipeline = Pipeline(steps = [('columntransformers', encoders),
                             ('model', rf)],
                    verbose=True)

# criação do f1_score usando average 'micro'
f1_score_micro = make_scorer(f1_score, average='micro')

# Gridsearch - pega o pipeline, os parametros do modelo, a 'logica' de divisão kfold e treina modelos, 
# avaliando pelo f1 score e no final, retreinando o melhor modelo.
grid_rf = GridSearchCV(pipeline, params_rf, 
                      cv = cv_kfold, 
                      scoring = f1_score_micro,
                      return_train_score = True,
                      refit=True,
                      verbose = 3)

In [21]:
# transforma y em 0 e 1 
target_encod = LabelBinarizer() 
y_train = target_encod.fit_transform(y_train)

In [22]:
# fit na grid
# .ravel() devido à warning
grid_rf.fit(X_train,y_train.ravel())

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[ColumnTransformer] ... (1 of 2) Processing onehotencod, total=   0.0s
[ColumnTransformer] ..... (2 of 2) Processing remainder, total=   0.0s
[Pipeline]  (step 1 of 2) Processing columntransformers, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   2.8s
[CV 1/5] END model=RandomForestClassifier(random_state=2187), model__class_weight=None, model__criterion=entropy, model__max_depth=None, model__n_estimators=100;, score=(train=0.990, test=0.871) total time=   2.9s
[ColumnTransformer] ... (1 of 2) Processing onehotencod, total=   0.0s
[ColumnTransformer] ..... (2 of 2) Processing remainder, total=   0.0s
[Pipeline]  (step 1 of 2) Processing columntransformers, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   2.7s
[CV 2/5] END model=RandomForestClassifier(random_state=2187), model__class_weight=None, model__criterion=entropy, model__max_depth=None, model__n_estimators

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=2187, shuffle=True),
             estimator=Pipeline(steps=[('columntransformers',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehotencod',
                                                                         OneHotEncoder(),
                                                                         ['country',
                                                                          'relationship_with_head',
                                                                          'marital_status',
                                                                          'education_level',
                                                                          'cellphone_access',
                                                                          'job_type',
                                                 

In [23]:
# quais os parametros do melhor modelo encontrado
grid_rf.best_params_

{'model': RandomForestClassifier(criterion='entropy', max_depth=10, random_state=2187),
 'model__class_weight': None,
 'model__criterion': 'entropy',
 'model__max_depth': 10,
 'model__n_estimators': 100}

In [24]:
# qual o score do melhor modelo
grid_rf.best_score_

0.884488900987915

In [25]:
# checando o score do modelo encontrado pelo gridsearch na validação
y_val = target_encod.transform(y_val)
grid_rf.score(X_val,y_val)

0.8821195806177388

# 6.0. Submission

In [26]:
teste = pd.read_csv('/kaggle/input/inclusao-financeira-na-africa/test.csv')

# drop same columns as the train dataset
# teste_clean = teste.drop(['uniqueid', 'uid'], axis=1)
teste_clean_dropped = teste.drop(['uniqueid', 'uid'], axis=1)

teste_clean_dropped.head()

,country,year,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Rwanda,2016,Rural,Yes,7,40,Male,Head of Household,Married/Living together,No formal education,Informally employed
1,Rwanda,2016,Rural,Yes,3,24,Male,Child,Single/Never Married,Secondary education,Farming and Fishing
2,Rwanda,2016,Urban,Yes,3,25,Female,Spouse,Married/Living together,Primary education,Farming and Fishing
3,Tanzania,2017,Urban,Yes,1,35,Female,Head of Household,Married/Living together,Primary education,Self employed
4,Rwanda,2016,Rural,Yes,3,60,Male,Head of Household,Married/Living together,Primary education,Farming and Fishing


In [27]:
# predict com o modelo treinado
bank_account = grid_rf.predict(teste_clean_dropped)

# dataframe com as predições
submission = pd.DataFrame(bank_account, columns=['bank_account'])

# transforming the prediction data
submission_words = submission.bank_account.apply(lambda x: 'Yes' if x==1 else 'No' )

# concatenando com o teste
submission2 = pd.concat([teste.uid, submission_words],axis=1)

In [28]:
submission2.head()

,uid,bank_account
0,Rwanda_uniqueid_625,No
1,Rwanda_uniqueid_1561,No
2,Rwanda_uniqueid_4806,No
3,Tanzania_uniqueid_4902,No
4,Rwanda_uniqueid_980,No


In [29]:
submission2.to_csv('../submission4.csv',index=False) #gridsearch